# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:



In [1]:
%%time
with open('checkins.dat') as input_file:
    csv_lines = []
    for line in input_file:
        # strip - Удаление пробельных символов в начале и в конце строки
        csv_line = [x.strip() for x in line.split('|')]
        if len(csv_line) == 6 and csv_line[3] and csv_line[4]:
            csv_lines.append(csv_line)

Wall time: 4.31 s


In [2]:
import csv

In [3]:
with open('checkins.csv', 'w') as output_file:
    writer = csv.writer(output_file)
    writer.writerows(csv_lines)

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [4]:
import numpy as np
import pandas as pd

In [5]:
checkins_data = pd.read_csv('checkins.csv', header = 0)

In [6]:
checkins_data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [7]:
len(checkins_data)

396634

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

In [8]:
checkins_data_сut = checkins_data.values[0:100000, 3:5]

In [9]:
len(checkins_data_сut)

100000

In [10]:
checkins_data_сut

array([[38.895111799999995, -77.0363658],
       [33.800745, -84.41051999999999],
       [45.5234515, -122.6762071],
       ...,
       [29.762884399999997, -95.3830615],
       [32.802955, -96.76992299999999],
       [37.7749295, -122.4194155]], dtype=object)

In [11]:
from sklearn.cluster import MeanShift

In [12]:
%%time
clusters = MeanShift(bandwidth = 0.1).fit(checkins_data_сut)

Wall time: 5min 14s


In [13]:
labels = clusters.labels_
labels

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)

In [14]:
cluster_centers = clusters.cluster_centers_
cluster_centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [15]:
labels_unique =np.unique(labels)
len(labels_unique)

3231

In [16]:
cluster_centers[1]

array([  33.44943805, -112.00213969])

In [17]:
d = {}
for label in labels:
    if label not in d.keys():
        d[label] = 1
    else:
        d[label] += 1

In [18]:
count = 0
for key in d.keys():
    if d[key] > 15:
        count += 1
print(count)

592


In [19]:
clusters_select = np.ndarray(shape =(count, 2))
clusters_select

array([[4.94065646e-324, 2.52961611e-321],
       [6.95243988e-310, 0.00000000e+000],
       [1.68476385e-321, 8.45620415e-307],
       ...,
       [1.21372167e-319, 9.96135155e-320],
       [1.01184644e-320, 9.96233968e-320],
       [1.46633743e-319, 3.69561103e-321]])

In [20]:
i = 0
j = 0
while i < len(cluster_centers):
    if d[i] > 15:
        clusters_select[j] = cluster_centers[i]
        j += 1
    i += 1

In [21]:
len(clusters_select)

592

In [22]:
clusters_select[0]

array([ 40.7177164 , -73.99183542])

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [23]:
offices = np.ndarray(shape=(6,2))
offices[0] = np.array([33.751277, -118.188740])
offices[1] = np.array([25.867736, -80.324116])
offices[2] = np.array([51.503016, -0.075479])
offices[3] = np.array([52.378894, 4.885084])
offices[4] = np.array([39.366487, 117.036146])
offices[5] = np.array([-33.868457, 151.205134])
offices

array([[ 3.37512770e+01, -1.18188740e+02],
       [ 2.58677360e+01, -8.03241160e+01],
       [ 5.15030160e+01, -7.54790000e-02],
       [ 5.23788940e+01,  4.88508400e+00],
       [ 3.93664870e+01,  1.17036146e+02],
       [-3.38684570e+01,  1.51205134e+02]])

In [24]:
def distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [25]:
distance(offices[0], clusters_select[123])

20.892897123756782

In [26]:
answer_index = 0
min_dist = 0
i = 0
while i < len(clusters_select):
    distances = [distance(xx, clusters_select[i]) for xx in offices]
    if min_dist == 0:
        min_dist = min(distances)
        answer_index = i
    else:
        if min_dist > min(distances):
            min_dist = min(distances)
            answer_index = i
    i += 1

In [27]:
answer_index

417

In [28]:
min_dist

0.007834758163107856

In [29]:
clusters_select[answer_index]

array([-33.86063043, 151.20477593])

In [30]:
def write_answer(center):
    with open("3_1_.txt", "w") as f:
        f.write(str(center[0]) + ' ' + str(center[1]))

In [31]:
write_answer(clusters_select[answer_index])